In [16]:
!pip install opencv-python

In [17]:
import numpy as np
import cv2 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from collections import Counter
import skimage
import scipy.stats as stats

In [18]:
def find_vector_set(diff_image, new_size):
    i = 0
    j = 0
    vector_set = np.zeros((int(new_size[0] * new_size[1] / 25), 25))
    while i < vector_set.shape[0]:
        while j < new_size[0]:
            k = 0
            while k < new_size[1]:
                block   = diff_image[j:j+5, k:k+5]
                feature = block.ravel()
                vector_set[i, :] = feature
                k = k + 5
            j = j + 5
        i = i + 1
 
    mean_vec   = np.mean(vector_set, axis = 0)
    vector_set = vector_set - mean_vec
    print(np.size(vector_set,0),np.size(vector_set,1))
    return vector_set, mean_vec

In [19]:
def find_FVS(EVS, diff_image, mean_vec, new):
 
    i = 2
    feature_vector_set = []
 
    while i < new[0] - 2:
        j = 2
        while j < new[1] - 2:
            block = diff_image[i-2:i+3, j-2:j+3]
            feature = block.flatten()
            feature_vector_set.append(feature)
            j = j+1
        i = i+1
 
    FVS = np.dot(feature_vector_set, EVS)
    FVS = FVS - mean_vec
    print("\nfeature vector space size", FVS.shape)
    return FVS
    

In [20]:
def clustering(FVS, components, new):
 
    kmeans = KMeans(components, verbose = 0)
    kmeans.fit(FVS)
    output = kmeans.predict(FVS)
    count  = Counter(output)
 
    least_index = min(count, key = count.get)
    change_map  = np.reshape(output,(new[0] - 4, new[1] - 4))
    return least_index, change_map

In [21]:
if __name__ == "__main__":
    a = "stacking_output1.png"
    b = "stacking_output2.png"

In [22]:
img1 = cv2.imread(a)
img2 = cv2.imread(b)

In [23]:
img1=np.array(img1)
img2=np.array(img2)

In [24]:
np.shape(img1)

(831, 1251, 3)

In [25]:
np.shape(img2)

(831, 1251, 3)

In [26]:
image1=img1[:,:,1]
image2=img2[:,:,1]

In [27]:
new_size = np.asarray(img1.shape) / 5
new_size = new_size.astype(int) * 5

In [28]:
image1 = cv2.resize(image1, (new_size[1],new_size[0])).astype(np.int16)
image2 = cv2.resize(image2, (new_size[1],new_size[0])).astype(np.int16)

In [29]:
diff_image = abs(image1 - image2)
diff_image

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [30]:
vector_set, mean_vec = find_vector_set(diff_image, new_size)
pca  = PCA()
pca.fit(vector_set)
EVS = pca.components_

41500 25


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:501: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


In [31]:
FVS  = find_FVS(EVS, diff_image, mean_vec, new_size)
k = 3
least_index, change_map = clustering(FVS, k, new_size)
 
change_map[change_map == least_index] = 255
change_map[change_map != 255] = 0
 
change_map = change_map.astype(np.uint8)


kernel1    = np.asarray(((0,0,1,0,0),
                         (0,1,1,1,0),
                         (1,1,1,1,1),
                         (0,1,1,1,0),
                         (0,0,1,0,0)), dtype=np.uint8)


feature vector space size (1029196, 25)


In [32]:
kernel2=np.asarray(((0,0,0,1,0,0,0),
                   (0,0,1,1,1,0,0),
                   (0,1,1,1,1,1,0),
                   (1,1,1,1,1,1,1),
                   (0,1,1,1,1,1,0),
                   (0,0,1,1,1,0,0),
                   (0,0,0,1,0,0,0)),dtype=np.uint8)

cleanChangeMap = cv2.erode(change_map,kernel1)
clean=cv2.dilate(cleanChangeMap,kernel1)

In [33]:
np.shape(img1)

(831, 1251, 3)

In [34]:
np.shape(img2)

(831, 1251, 3)

In [35]:
cleanChangeMap_New=cv2.resize(cleanChangeMap,(np.size(img1,1),np.size(img1,0))).astype(np.int16)

In [36]:
np.shape(cleanChangeMap_New)

(831, 1251)

In [37]:
cv2.imwrite('a_erode.png',cleanChangeMap)
cv2.imwrite('a_without.png',change_map)
cv2.imwrite('a_dilate.png',clean)

True

In [ ]:
!pip install image_similarity_measures

In [43]:
from image_similarity_measures.quality_metrics import rmse, psnr

In [48]:
a = "stacking_output1.png"
b = "stacking_output2.png"
y = "a_erode (1).png"

In [49]:
in_img1 = cv2.imread(y)
in_img2 = cv2.imread(b)
out_rmse = rmse(in_img1, in_img2)
out_psnr = psnr(in_img1, in_img2)

In [50]:
print(out_rmse)
print(out_psnr)

0.040098414
27.937459980754426


## RSME:- higher the value, lower the quality
## PSNR:- higher the value, higher the quality